In [ ]:
%matplotlib widget
from matplotlib import pyplot as plt
import matplotlib as mp
from matplotlib import cm
import numpy as np
import seaborn as sns
import cmocean as cmo
from cmocean import plots

In [ ]:
from lotr.plotting.colors import get_n_colors
from matplotlib.colors import ListedColormap

viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(0, 1, 256))
pink = np.array([248/256, 24/256, 148/256, 1])
newcolors[:25, :] = pink
newcmp = ListedColormap(newcolors)

newcmp = ListedColormap(newcolors)

In [ ]:
f, ax = plt.subplots(figsize=(3, 3))
plots.test(cmo.cm.phase, ax=ax)
plt.ylim(0, 100)
plt.tight_layout()

In [ ]:
#60, 40
# network_phase: lum=45, sat=70, hshift=90
# pc_angle
phase_cm = ListedColormap(get_n_colors(1000, lum=60, sat=45, hshift=90) / 255)
f, ax = plt.subplots(figsize=(3, 3))
plots.test(phase_cm, ax=ax)
plots.test(cmo.cm.phase, ax=ax)
plt.ylim(0, 100)
plt.tight_layout()

In [ ]:
plt.figure()
plt.imshow(np.random.rand(100, 20), cmap=phase_cm)